

#   This file demonstrates the migration process using the video dataset.

## Input data: define the ABAC policies for video dataset

In [1]:
#define ABAC policy (video)


U_v=[
   
{'uid':'child1r','ageGroup':'child','memberType':'regular'},
{'uid':'child2r','ageGroup':'child','memberType':'regular'},
{'uid':'teen1r','ageGroup':'teen','memberType':'regular'},
{'uid':'teen2r','ageGroup':'teen','memberType':'regular'},
{'uid':'adult1r','ageGroup':'adult','memberType':'regular'},
{'uid':'adult2r','ageGroup':'adult','memberType':'regular'}, 

{'uid':'child1p','ageGroup':'child','memberType':'premium'},
{'uid':'child2p','ageGroup':'child','memberType':'premium'},
{'uid':'teen1p','ageGroup':'teen','memberType':'premium'},
{'uid':'teen2p','ageGroup':'teen','memberType':'premium'},
{'uid':'adult1p','ageGroup':'adult','memberType':'premium'},
{'uid':'adult2p','ageGroup':'adult','memberType':'premium'},  
]

R_v=[

{'rid':'the_lion_king','rating':'G','videoType':'old'},
{'rid':'toy_story','rating':'G','videoType':'old'},
{'rid':'star_wars','rating':'PG','videoType':'old'},
{'rid':'jaws','rating':'PG','videoType':'old'},
{'rid':'brave_heart','rating':'R','videoType':'old'},
{'rid':'my_cousin_vinny','rating':'R','videoType':'old'},


{'rid':'happy_feet','rating':'G','videoType':'new'},
{'rid':'happy_feet_2','rating':'G','videoType':'new'},
{'rid':'cars','rating':'G','videoType':'new'},
{'rid':'brave','rating':'PG','videoType':'new'},
{'rid':'life_of_pi','rating':'PG','videoType':'new'},
{'rid':'flight','rating':'R','videoType':'new'},
{'rid':'cloud_atlas','rating':'R','videoType':'new'}   
]
UA_v=['ageGroup','memberType','uid']
RA_v=['rid','rating','videoType']
O_v=['view']
RU_v=[
  
# premium members of all ages can view movies rated G.
{'expressions':[{'eu':{'memberType':[('is','premium')]},'er':{'rating':[('is','G')]}}],'operations':['view'],'decision':'allow'}, 
# premium teens can view movies rated PG.
{'expressions':[{'eu':{'memberType':[('is','premium')],'ageGroup':[('is','teen')]},'er':{'rating':[('is','PG')]}}],'operations':['view'],'decision':'allow'}, 
# premium adults can view movies with all ratings.
{'expressions':[{'eu':{'memberType':[('is','premium')],'ageGroup':[('is','adult')]},'er':{}}],'operations':['view'],'decision':'allow'}, 
# members of all ages can view old movies rated G.
{'expressions':[{'eu':{},'er':{'rating':[('is','G')],'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 

# teens can view old movies rated PG.
{'expressions':[{'eu':{'ageGroup':[('is','teen')]},'er':{'rating':[('is','PG')],'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 
# adults can view old movies with all ratings.
{'expressions':[{'eu':{'ageGroup':[('is','adult')]},'er':{'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 

]
def dU(user,attribute):
  if attribute in user.keys():
    return user[attribute]

def dR(resource,attribute):
  if attribute in resource.keys():
    return resource[attribute]
ABAC_video=(U_v,R_v,O_v,UA_v,RA_v,dU,dR,RU_v)

## Define the ABAC policy checker
Because in video dataset, only equality is considered, we only consider the constraint "is" in this demo.

In [24]:
def satisfied(u,r,expressions):
  for expression in expressions:
    eu=expression['eu']
    er=expression['er']
    #for all the keys in eu, get the constraint of this key
    #if u does not satisfy this constraint, return false\
    if ( len(eu.keys()) != 0 and u != None):
      for key in eu.keys():
        if(dU(u,key) == None):
            return False
        for (op,c) in eu[key]:
          # u does not have this key
          if(dU(u,key) == None):
            return False
        #op,c=eu[key]
        #print(op,c)
          if op == 'is' and  dU(u,key) != c:
            return False

    #for all the keys in er get the constraint of the key
    if ( len(er.keys()) != 0 and r !=None):
      for key in er.keys():
        if dR(r,key)==None:
          #  print(dR(r,key),key,op,c)
            return False
        for (op,c) in er[key]:
        #print(op,c)
        #op,c=er[key]
          if op == 'is' and dR(r,key)!=c:
            return False
  
  return True


def check_ABAC_policy(request,abac_policy):
  u,r,o =request
  _,_,_,_,_,_,_,rules = abac_policy
  for rule in rules:
    if o in rule['operations']:
      if satisfied(u,r,rule['expressions']):
        return True
  return False

# check the ABAC policy 
number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_ABAC_policy((u,r,o),ABAC_video)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For ABAC policy:\nNumber of allowed requests:{number_allow}\nNumber of denied request:{number_deny}')

#print(f'{check_ABAC_policy((U[6],R[7],"readMyScores"),ABAC_university)}\n{U[6]}\n{R[7]}')
#print(f'{check_ABAC_policy((U[3],R[5],"addScore"),ABAC_university)}\n{U[3]}\n{R[5]}')
#print(f'Loss at epo {epoch}: {losses/len(training_data)}')



For ABAC policy:
Number of allowed requests:78
Number of denied request:78


## Calculate the number of user constraints and the number of permissions(resource constraint,operation)

In [25]:
from numpy import *
# Calculate the number of user constraints
# Calculate the number of permissions(resource constraint,operation)
UC=[]
Permission=[]
for rule in RU_v:
  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    if uc not in UC:
      UC.append(uc)
  if (resource_cons,operations) not in Permission:
    Permission.append((resource_cons,operations))
n_uc=len(UC)
n_p=len(Permission)
print(f'Number of user constraints:{n_uc}\nNumber of Permission sets:{n_p}')
print(f'User constraints:')
print(UC)
print(f'Permission sets')
print(Permission)


Number of user constraints:3
Number of Permission sets:6
User constraints:
[{'memberType': [('is', 'premium')]}, {'ageGroup': [('is', 'teen')]}, {'ageGroup': [('is', 'adult')]}]
Permission sets
[({'rating': [('is', 'G')]}, ['view']), ({'rating': [('is', 'PG')]}, ['view']), ({}, ['view']), ({'rating': [('is', 'G')], 'videoType': [('is', 'old')]}, ['view']), ({'rating': [('is', 'PG')], 'videoType': [('is', 'old')]}, ['view']), ({'videoType': [('is', 'old')]}, ['view'])]


In [26]:
# Return the place of the item in the list
# For example get_item_id(3,[2,3,5,4]) will return 1
def get_item_id(item,list):
  id = 0
  while(id<len(list)):
    if item == list[id]:
      return id
    id = id + 1
  return None

def cover(ma,mb):# ma cover mb
  if ma.shape[1] != mb.shape[1]:
    return False
  for i in range(ma.shape[1]):
    if mb[0,i]==1 and ma[0,i]==0:
      return False


  return True

## Get the permission-constraint matrix and the role-permission matrix

In [27]:
# permission-constraint matrix
pc=mat(zeros((n_p,n_uc)))
for rule in RU_v:
  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  p_id=get_item_id((resource_cons,operations),Permission)
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    c_id=get_item_id(uc,UC)
    if(c_id==None):
      continue
    pc[p_id,c_id]=1
print(f'The pc matrix is\n {pc}')
rp=mat(diag(array(ones(n_p))))
for i in range(n_p-1):
  for j in range(i+1,n_p):
    if (cover(pc[i,],pc[j,])):
      rp[i,j]=1
    if (cover(pc[j,],pc[i,])):
      rp[j,i]=1

print(f'The rp matrix is\n {rp}')

The pc matrix is
 [[1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
The rp matrix is
 [[1. 0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1. 0.]
 [1. 0. 1. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1.]]


## Calculate the user-role assignment and role-permission assignment according to the pc and rp matrix.

In [28]:
def rest_of(l,r):
  vector = mat(zeros((1,n_uc)))
  for role in l:
    if role != r:
      vector = vector + rc[role,]
  for i in range(n_uc):
    if vector[0,i]>0:
      vector[0,i]=1
  return vector

import copy
# User role assignment 
n_r=n_p
rc=pc
URA = dict()
print(f'The user-role assignment is:')
for u in U_v:
  URA[u['uid']]=[]
  uv=mat(ones((1,n_uc)))
  for cid in range(n_uc):
    expressions=[{'eu':UC[cid],'er':{}}]
    if satisfied(u,None,expressions)==False:
      uv[0,cid]=0
  for rid in range(n_r):
    if cover(uv,rc[rid,]):
      URA[u['uid']].append(rid)

# remove hierarchy
  for ri in URA[u['uid']]:
    rest = URA[u['uid']].copy()
    rest.remove(ri)

    for rj in rest:
      if cover(rc[rj,],rc[ri,]) and ri in URA[u['uid']]:
        URA[u['uid']].remove(ri)
#remove redundant
  for r in URA[u['uid']]:
    if len(URA[u['uid']])==1:
      break
    if cover(rest_of(URA[u['uid']],r),rc[r,]):
      URA[u['uid']].remove(r)

  ura =  URA[u['uid']]

  print(f'User {u}, role: {ura}')
#print(f'The user-role assignment:  {URA}')
 # print(f'The role-permission assignment:  {RPA}')

# generate the role-heirarchy
RH = []
for i in range(0,n_r-1):
  for j in range(i+1,n_r):
    if (cover(pc[i,],pc[j,])):
      RH.append((i,j))
    if (cover(pc[j,],pc[i,])):
      RH.append((j,i))    
print(f'\nThe role-hierarchy relationship:  {RH}')
# generate the role-permission assignment
RPA = dict()
for ro in range(n_r):
  RPA['role'+str(ro)]=[]
  for r in R_v:
    for o in O_v:
      for pid in range(n_p):
        if rp[ro,pid]==1:
          (er,operations) = Permission[pid]
          #print(er,r)
          expressions=[{'eu':{},'er':er}]
          #print(satisfied(None,r,expressions))
          if satisfied(None,r,expressions) and o in operations:
            if((r,o) not in RPA['role'+str(ro)]):
              RPA['role'+str(ro)].append((r,o))

print(f'\nThe role-permission assignment is:')
for ro in range(n_r):
  p=RPA['role'+str(ro)]
  print(f'Role：{ro}, Number of permissions:{len(p)},  Permissions: {p}')

The user-role assignment is:
User {'uid': 'child1r', 'ageGroup': 'child', 'memberType': 'regular'}, role: [3]
User {'uid': 'child2r', 'ageGroup': 'child', 'memberType': 'regular'}, role: [3]
User {'uid': 'teen1r', 'ageGroup': 'teen', 'memberType': 'regular'}, role: [4]
User {'uid': 'teen2r', 'ageGroup': 'teen', 'memberType': 'regular'}, role: [4]
User {'uid': 'adult1r', 'ageGroup': 'adult', 'memberType': 'regular'}, role: [5]
User {'uid': 'adult2r', 'ageGroup': 'adult', 'memberType': 'regular'}, role: [5]
User {'uid': 'child1p', 'ageGroup': 'child', 'memberType': 'premium'}, role: [0]
User {'uid': 'child2p', 'ageGroup': 'child', 'memberType': 'premium'}, role: [0]
User {'uid': 'teen1p', 'ageGroup': 'teen', 'memberType': 'premium'}, role: [1]
User {'uid': 'teen2p', 'ageGroup': 'teen', 'memberType': 'premium'}, role: [1]
User {'uid': 'adult1p', 'ageGroup': 'adult', 'memberType': 'premium'}, role: [2]
User {'uid': 'adult2p', 'ageGroup': 'adult', 'memberType': 'premium'}, role: [2]

The ro

In [29]:
# check RBAC policy
def check_RBAC_policy(request,rp,URA):
  u,r,o =request
  for role_id in URA[u['uid']]:
    for pid in range(n_p):
      if rp[role_id,pid]==1:
        (er,operations) = Permission[pid]
        expressions=[{'eu':{},'er':er}]
        if satisfied(u,r,expressions) and o in operations:
          return True


  return False


In [30]:
# Check the RBAC
# check the RBAC policy 

number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_RBAC_policy((u,r,o),rp,URA)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')


print(f'For RBAC policy:\nNumber of allowed requests:{number_allow}\nNumber of denied request:{number_deny}')

For RBAC policy:
Number of allowed requests:78
Number of denied request:78


## Evaluate the correctness of the migration

In [32]:

# check the RBAC policy 
number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_RBAC_policy((u,r,o),rp,URA)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')


print(f'For RBAC policy:\nNumber of allowed requests:{number_allow}\nNumber of denied request:{number_deny}')

# check the ABAC policy 
number_allow=0
number_deny=0
m= []
for u in U_v:
  l = []
  for r in R_v:
    for o in O_v:
      if(check_ABAC_policy((u,r,o),ABAC_video)==True):
        l.append(1)
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        l.append(0)
        #print(f'{u}\n{r}\n{o}\n \n')
  m.append(l)


print(f'For ABAC policy:\nNumber of allowed requests:{number_allow}\nNumber of denied request:{number_deny}')
#for i in m:
#  print(i)
#number_allow=0
#number_deny=0

correct=0
for u in U_v:
 
  for r in R_v:
    for o in O_v:
      if (check_ABAC_policy((u,r,o),ABAC_video)==True and check_RBAC_policy((u,r,o),rp,URA)==False):
        print(f'TF{u} {r} {o}')
      elif (check_ABAC_policy((u,r,o),ABAC_video)==False and check_RBAC_policy((u,r,o),rp,URA)==True):
        print(f'FT{u} {r} {o}')
        #number_allow= number_allow+1
      elif (check_ABAC_policy((u,r,o),ABAC_video)==True and check_RBAC_policy((u,r,o),rp,URA)==True):
        correct = correct +1
      elif (check_ABAC_policy((u,r,o),ABAC_video)==False and check_RBAC_policy((u,r,o),rp,URA)==False):
        correct = correct +1
        #number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'\nNumber of correctness:{correct}')


For RBAC policy:
Number of allowed requests:78
Number of denied request:78
For ABAC policy:
Number of allowed requests:78
Number of denied request:78

Number of correctness:156
